In [49]:
import pandas as pd
import numpy as np
import os
from keras.models import Sequential
from keras.layers import Dense, Activation,LSTM,Dropout
from keras.optimizers import Adam
import tensorflow as tf



    
def sliceWindow(data,target,step): 
    X,y,temp=[],[],[]
    temp.append(target)
    for i in range(0,len(np.array(data))-2*step,1):  
        end=i+step       
        oneX,oney=np.array(data)[i:end,:],np.array(data[temp])[end:end+step, :]  
        X.append(oneX)  
        y.append(oney)  
    return np.array(X),np.array(y)  



def dataSplit(dataset,target,step,ratio=0.80):  
    datasetX,datasetY=sliceWindow(dataset,target,step)  
    train_size=int(len(datasetX)*ratio)  
    X_train,y_train=datasetX[0:train_size,:],datasetY[0:train_size,:] 
    X_test,y_test=datasetX[train_size:len(datasetX),:],datasetY[train_size:len(datasetX),:]  
    X_train=X_train.reshape(X_train.shape[0],step,-1)  
    X_test=X_test.reshape(X_test.shape[0],step,-1)  
    y_train = y_train.reshape(y_train.shape[0],-1,step)
    y_test = y_test.reshape(y_test.shape[0],-1,step)
    print('X_train.shape: ',X_train.shape)  
    print('X_test.shape: ',X_test.shape)  
    print('y_train.shape: ',y_train.shape)  
    print('y_test.shape: ',y_test.shape)  
    return X_train,X_test,y_train,y_test  


print('============获取数据=================')
path = os.getcwd()
test = pd.read_csv(path+'\Quant_Public_Data_March\预测目标.测试集.csv')
train = pd.read_csv(path+'\Quant_Public_Data_March\预测目标.训练集.csv')
mkt_data = pd.read_csv(path+'\Quant_Public_Data_March\市场数据.训练集.日.csv')
fin_monthly_data = pd.read_csv(path+'\Quant_Public_Data_March\经济数据.训练集.月.csv')
fin_seasonly_data = pd.read_csv(path+'\Quant_Public_Data_March\经济数据.训练集.季.csv')
fin_yearly_data = pd.read_csv(path+'\Quant_Public_Data_March\经济数据.训练数据.年.csv')
fin_est_data = pd.read_csv(path+'\Quant_Public_Data_March\房地产数据.训练集.月.csv')


# 将非日数据转化为日数据
fin_est_data[['biz_date']] = fin_est_data['biz_month']+'-01'
fin_seasonly_data[['biz_date']] = fin_seasonly_data['biz_season']+'-01'
fin_monthly_data[['biz_date']] = fin_monthly_data['biz_month']+'-01'

del fin_est_data['biz_month']
del fin_seasonly_data['biz_season']
del fin_monthly_data['biz_month']



# 所有字段拼接起来
origin_train_data = pd.merge(pd.merge(pd.merge(pd.merge(train,mkt_data,on = ['biz_date'],how='outer'),
                                       fin_est_data,on = ['biz_date'],how='outer'),
                              fin_seasonly_data,on = ['biz_date'],how='outer'),
                     fin_monthly_data,on = ['biz_date'],how='outer')

del origin_train_data['Unnamed: 5']

final_pre = []
target_label = [c for c in origin_train_data.columns if '_10y' in c]

for target in target_label:
    train_data = origin_train_data
    print('========='+target +'===========')

    train_data = train_data[train_data[target].notna()]
    train_data = train_data.fillna(0)
    train_data = train_data.sort_values(by='biz_date') 
    train_data= train_data.set_index('biz_date')
    
    print('============保存'+target+'数据=================')
    
    
    train_data.to_csv(target+'.csv',index=True,header=True) 

    print('============数据分割=================')

    X_train,X_test,y_train,y_test  = dataSplit(train_data,target,step=30)

    # 创建模型

    print('============创建模型=================')
    model = Sequential()

    model.add(LSTM(64, input_shape=(X_train.shape[1],X_train.shape[2]),activation='sigmoid', return_sequences=True, dropout=0.01))
    model.add(LSTM(128, activation='sigmoid', return_sequences=True, dropout=0.01))
    model.add(Dropout(rate=0.01))
    model.add(LSTM(128, activation='sigmoid', dropout=0.01))
    model.add(Dense(30))
    model.compile(loss='mae', optimizer=Adam(lr=0.002, decay=0.01))

    print(model.summary())

    # 训练模型
    print('============训练模型=================')

    tf.config.experimental_run_functions_eagerly(True)
    history = model.fit(X_train, y_train, epochs=50, batch_size=100, validation_data=(X_test, y_test), shuffle=False)
    model.save(target + ".h5")

    print('=============预测数据================')
    pre_train = np.array(train_data.tail(30))
    pre_train = pre_train.reshape(1,30,44)  
    y_pre=model.predict(pre_train,verbose=0)
    print(y_pre)
    final_pre.append(y_pre)

pd.DataFrame(final_pre).to_csv('final_pre.csv',index=False,header=True) 

============获取数据=================
=========treasury_bond_rate_10y===========
============保存treasury_bond_rate_10y数据=================
============数据分割=================
X_train.shape:  (3814, 30, 44)
X_test.shape:  (954, 30, 44)
y_train.shape:  (3814, 1, 30)
y_test.shape:  (954, 1, 30)
============创建模型=================
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 30, 64)            27904     
_________________________________________________________________
lstm_13 (LSTM)               (None, 30, 128)           98816     
_________________________________________________________________
dropout_4 (Dropout)          (None, 30, 128)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 128)               131584    
_________________________________________________________________

C:\Users\Administrator\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\data\ops\dataset_ops.py:3503: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  warnings.warn(


39/39 [==============================] - 18s 458ms/step - loss: 1.8235 - val_loss: 0.3855
Epoch 2/50
39/39 [==============================] - 17s 426ms/step - loss: 0.5993 - val_loss: 0.4202
Epoch 3/50
39/39 [==============================] - 16s 422ms/step - loss: 0.6099 - val_loss: 0.4301
Epoch 4/50
39/39 [==============================] - 17s 439ms/step - loss: 0.6048 - val_loss: 0.3850
Epoch 5/50
39/39 [==============================] - 18s 473ms/step - loss: 0.5895 - val_loss: 0.3750
Epoch 6/50
39/39 [==============================] - 16s 415ms/step - loss: 0.5825 - val_loss: 0.3662
Epoch 7/50
39/39 [==============================] - 16s 417ms/step - loss: 0.5770 - val_loss: 0.3604
Epoch 8/50
39/39 [==============================] - 16s 415ms/step - loss: 0.5733 - val_loss: 0.3560
Epoch 9/50
39/39 [==============================] - 16s 417ms/step - loss: 0.5706 - val_loss: 0.3528
Epoch 10/50
39/39 [==============================] - 16s 414ms/step - loss: 0.5682 - val_loss: 0.3504


39/39 [==============================] - 19s 490ms/step - loss: 0.6565 - val_loss: 0.4872
Epoch 16/50
39/39 [==============================] - 19s 493ms/step - loss: 0.6552 - val_loss: 0.4863
Epoch 17/50
39/39 [==============================] - 19s 487ms/step - loss: 0.6540 - val_loss: 0.4855
Epoch 18/50
39/39 [==============================] - 19s 493ms/step - loss: 0.6529 - val_loss: 0.4848
Epoch 19/50
39/39 [==============================] - 19s 489ms/step - loss: 0.6519 - val_loss: 0.4842
Epoch 20/50
39/39 [==============================] - 19s 489ms/step - loss: 0.6511 - val_loss: 0.4838
Epoch 21/50
39/39 [==============================] - 19s 489ms/step - loss: 0.6504 - val_loss: 0.4834
Epoch 22/50
39/39 [==============================] - 19s 478ms/step - loss: 0.6497 - val_loss: 0.4829
Epoch 23/50
39/39 [==============================] - 18s 452ms/step - loss: 0.6491 - val_loss: 0.4825
Epoch 24/50
39/39 [==============================] - 17s 437ms/step - loss: 0.6485 - val_loss:

21/21 [==============================] - 10s 494ms/step - loss: 0.2894 - val_loss: 0.4363
Epoch 30/50
21/21 [==============================] - 10s 494ms/step - loss: 0.2892 - val_loss: 0.4366
Epoch 31/50
21/21 [==============================] - 10s 495ms/step - loss: 0.2891 - val_loss: 0.4369
Epoch 32/50
21/21 [==============================] - 10s 495ms/step - loss: 0.2889 - val_loss: 0.4372
Epoch 33/50
21/21 [==============================] - 10s 493ms/step - loss: 0.2888 - val_loss: 0.4374
Epoch 34/50
21/21 [==============================] - 10s 496ms/step - loss: 0.2886 - val_loss: 0.4376
Epoch 35/50
21/21 [==============================] - 11s 526ms/step - loss: 0.2885 - val_loss: 0.4378
Epoch 36/50
21/21 [==============================] - 11s 546ms/step - loss: 0.2884 - val_loss: 0.4380
Epoch 37/50
21/21 [==============================] - 11s 521ms/step - loss: 0.2883 - val_loss: 0.4382
Epoch 38/50
21/21 [==============================] - 11s 530ms/step - loss: 0.2881 - val_loss:

25/25 [==============================] - 13s 525ms/step - loss: 0.4310 - val_loss: 0.9397
Epoch 44/50
25/25 [==============================] - 13s 525ms/step - loss: 0.4308 - val_loss: 0.9396
Epoch 45/50
25/25 [==============================] - 13s 525ms/step - loss: 0.4306 - val_loss: 0.9395
Epoch 46/50
25/25 [==============================] - 13s 523ms/step - loss: 0.4304 - val_loss: 0.9394
Epoch 47/50
25/25 [==============================] - 13s 528ms/step - loss: 0.4303 - val_loss: 0.9394
Epoch 48/50
25/25 [==============================] - 13s 532ms/step - loss: 0.4301 - val_loss: 0.9393
Epoch 49/50
25/25 [==============================] - 13s 526ms/step - loss: 0.4300 - val_loss: 0.9392
Epoch 50/50
25/25 [==============================] - 13s 525ms/step - loss: 0.4298 - val_loss: 0.9391
=============预测数据================
[[5.2633967 5.263536  5.2634354 5.263477  5.263466  5.2627244 5.2622375
  5.262067  5.2612453 5.261221  5.2612305 5.2613034 5.2611704 5.2610903
  5.2612996 5.2611

AttributeError: 'list' object has no attribute 'to_csv'

In [50]:
pd.DataFrame(final_pre).to_csv('final_pre.csv',index=False,header=True) 

ValueError: Must pass 2-d input. shape=(5, 1, 30)

In [51]:
final_pre

[array([[3.513096 , 3.5142941, 3.514343 , 3.5143735, 3.5151334, 3.515776 ,
         3.5132782, 3.5153294, 3.5158706, 3.5163882, 3.5169113, 3.5152533,
         3.515257 , 3.5175238, 3.5189269, 3.5148804, 3.5132825, 3.5139997,
         3.513206 , 3.5135252, 3.5128067, 3.5112307, 3.511518 , 3.5128229,
         3.5129488, 3.5129685, 3.5124998, 3.5128512, 3.5128493, 3.5141168]],
       dtype=float32),
 array([[4.031071 , 4.0315194, 4.0321174, 4.0326886, 4.032132 , 4.032765 ,
         4.0339274, 4.034975 , 4.0357757, 4.0363503, 4.0363607, 4.0373836,
         4.0371566, 4.0377464, 4.0383544, 4.0383263, 4.040042 , 4.040608 ,
         4.0411234, 4.0411596, 4.040986 , 4.04148  , 4.0418873, 4.0425196,
         4.042497 , 4.042802 , 4.0432677, 4.0432787, 4.0433655, 4.0429463]],
       dtype=float32),
 array([[3.9088159, 3.898198 , 3.8853476, 3.9024794, 3.8846574, 3.9104345,
         3.9015927, 3.9088993, 3.8941846, 3.9061675, 3.8932173, 3.9125257,
         3.8899252, 3.892897 , 3.9299946, 3.899489